In [ ]:
%cd '/content/drive/MyDrive/eye_disease'

/content/drive/MyDrive/eye_disease


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.applications import inception_resnet_v2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import tensorflow as tf
import keras.backend as K

In [ ]:
def squeeze_excite_block(tensor, ratio=8):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = tensor.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(tensor)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)
    x = multiply([tensor, se])
    return x

In [ ]:

def channel_shuffle(x):
    height, width, channels = x.shape.as_list()[1:]
    channels_per_split = channels // 2
    x = K.reshape(x, [-1, height, width, 2, channels_per_split])
    x = K.permute_dimensions(x, (0,1,2,4,3))
    x = K.reshape(x, [-1, height, width, channels])
    return x

In [ ]:
def SK_Unit(input, M, r, L=32, stride=1):
    input_feature=K.int_shape(input)[3]
    #input_feature = input.get_shape().as_list()[3]
    d = max(int(input_feature / r), L)
    net = input

    prime_net=[]
    #with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu):
    for i in range(M):
       #net = Conv2D(input_feature, kernel_size=(3+i*2, 3+i*2), dilation_rate=2, padding='SAME')(net)
       net = Conv2D(input_feature, kernel_size=(3, 3), dilation_rate=1+i, padding='SAME')(net)
       net = BatchNormalization()(net)
       net = Activation('relu')(net)
       prime_net.append(net)
       if i == 0:
          fea_U = net
       else:
          fea_U = Add()([fea_U, net])
          #fea_U = tf.add(fea_U, net)

    #return fea_U

    gap = GlobalAveragePooling2D()(fea_U)
    gmp = GlobalMaxPooling2D()(fea_U)
    gap = Add()([gap, gmp])
    gap = Flatten()(gap)
    fc = Dense(d, activation='relu')(gap)
    fcs = fc
    for i in range(M):
        #att = Conv2D(input_feature, kernel_size=(1, 1))(fcs)
        att =Dense(input_feature)(fcs)
        att= Activation('sigmoid') (att)
        F_A = Multiply()([att,prime_net[i]])
        if i == 0:
          f_out = F_A
        else:
          f_out = Add()([f_out, F_A])

    return f_out

In [ ]:
base_model = inception_resnet_v2.InceptionResNetV2

base_model = base_model(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Comment this out if you want to train all layers
# for layer in base_model.layers:
#     layer.trainable = True

x = base_model.layers[-2].output

input_features=K.int_shape(x)[3]
int_layer=Conv2D(input_features, kernel_size=3)(x)
int_layer = channel_shuffle(int_layer)
int_layer = Activation('relu')(int_layer)
z=SK_Unit(int_layer,3,2)
x= squeeze_excite_block(x, 16)

#x = Flatten()(x)
#x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
predictions = Dense(8, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#for layer in model.layers[:freeze_layers]:
#    layer.trainable = True
#for layer in model.layers[freeze_layers:]:
#    layer.trainable = False

model.summary()

In [ ]:
defined_metrics = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
]

In [ ]:
from tensorflow.keras.optimizers import SGD
sgd = SGD(learning_rate=0.0007, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=defined_metrics)

In [ ]:
import numpy as np

x_train = np.load('odir_training_224.npy')
split_1 = int(0.8 * len(x_train))

train_dataset = x_train[:split_1]
test_dataset = x_train[split_1:]

y_train = np.load('odir_training_labels_224.npy')
split_2 = int(0.8 * len(y_train))

train_result = y_train[:split_2]
test_result = y_train[split_2:]


In [ ]:
#x_test = np.load('odir_testing_224.npy')
#y_test = np.load('odir_testing_labels_224.npy')

In [ ]:
batch_size = 16
history = model.fit(train_dataset, train_result,
                    batch_size=batch_size,
                    validation_data=(test_dataset, test_result),
                                    epochs=25, verbose=1, shuffle=True)

Epoch 1/25
954/954 [==============================] - 687s 666ms/step - loss: 0.2928 - accuracy: 0.8786 - precision: 0.6685 - recall: 0.3719 - auc: 0.8633 - val_loss: 0.3607 - val_accuracy: 0.8401 - val_precision: 0.4024 - val_recall: 0.1707 - val_auc: 0.7787
Epoch 2/25
954/954 [==============================] - 629s 660ms/step - loss: 0.2022 - accuracy: 0.9181 - precision: 0.8173 - recall: 0.5817 - auc: 0.9416 - val_loss: 0.3463 - val_accuracy: 0.8473 - val_precision: 0.4668 - val_recall: 0.2399 - val_auc: 0.8165
Epoch 3/25
954/954 [==============================] - 629s 659ms/step - loss: 0.1560 - accuracy: 0.9379 - precision: 0.8590 - recall: 0.6989 - auc: 0.9670 - val_loss: 0.3770 - val_accuracy: 0.8477 - val_precision: 0.4742 - val_recall: 0.2876 - val_auc: 0.8004
Epoch 4/25
954/954 [==============================] - 629s 659ms/step - loss: 0.1175 - accuracy: 0.9553 - precision: 0.8979 - recall: 0.7904 - auc: 0.9820 - val_loss: 0.4074 - val_accuracy: 0.8506 - val_precision: 0.4921

In [ ]:
test_predictions_baseline = model.predict(test_dataset)

In [ ]:
!mkdir 'ODIR-5K_Result_inception_se'

mkdir: cannot create directory ‘ODIR-5K_Result_inception_se’: File exists


In [ ]:
result_folder = 'ODIR-5K_Result_inception_se'

In [ ]:
from kappa_score import FinalScore
from predictions_writer import Prediction

In [ ]:
prediction_writer = Prediction(test_predictions_baseline, 400, result_folder)
prediction_writer.save()
prediction_writer.save_all(test_result)


In [ ]:
# show the final score
score = FinalScore(result_folder)
score.output()

Kappa score: 0.8146268656716419
F-1 score: 0.942875
AUC value: 0.9608205357142857
Final Score: 0.8697074671286426


In [ ]:
model.save('inception_se.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
